# Описание проекта
Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра». Нужно построить модель для задачи классификации, которая выберет подходящий тариф.

# Описание данных
Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц.
* сalls — количество звонков
* minutes — суммарная длительность звонков в минутах
* messages — количество sms-сообщений
* mb_used — израсходованный интернет-трафик в Мб
* is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt 

In [2]:
try:
    df = pd.read_csv('/datasets/users_behavior.csv')
except FileNotFoundError:
    df = pd.read_csv(r'C:\Users\ASUS\Desktop\Практикум\Data\users_behavior.csv')
    

In [3]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
df.sample(10)

,calls,minutes,messages,mb_used,is_ultra
2982,50.0,362.53,0.0,18476.21,0
1274,49.0,292.51,46.0,12751.50,0
2052,53.0,400.47,25.0,18307.71,0
759,35.0,182.51,103.0,25494.78,1
2002,34.0,274.53,13.0,5762.76,0
1712,84.0,513.10,21.0,16080.39,1
2123,34.0,252.87,26.0,24550.06,1
236,48.0,336.38,14.0,21902.41,0
1779,56.0,415.72,7.0,16819.71,0
1231,13.0,82.55,4.0,2890.46,1


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Поменяем тип данных в количестве звонков и сообщений на int

In [6]:
df['calls'] = df['calls'].astype('int')

In [7]:
df['messages'] = df['messages'].astype('int')

Проверим датасет на дубликаты

In [8]:
df.duplicated().sum()

0

## Разбейте данные на выборки

In [9]:
features = df.drop('is_ultra', axis = 1)

In [10]:
target = df['is_ultra']

Разобьем данные на три части: обучающую, валидационную и тестовую в соотношении 3:1:1. 

In [11]:
features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.2, random_state=12345)
features_valid, features_test, target_valid, target_test = train_test_split(features_valid, target_valid, test_size=0.2 , random_state=12345)

## Исследуйте модели

Для исследования исползуем 3 модели: Логистическая регрессия, Случайный лес и Дерево решений

### Обучим модель с использованием Логической регресии

In [12]:
# Обучение модели логистической регрессии
model_lr = LogisticRegression(random_state=12345)
model_lr.fit(features_train, target_train)
predictions_lr = model_lr.predict(features_valid)
accuracy_lr = accuracy_score(target_valid, predictions_lr)
print('Accuracy:', accuracy_lr)

Accuracy: 0.7587548638132295


### Обучим модель с использованием Случайного леса

In [13]:
best_result_rf = 0
best_depth_rf = 0
best_model_rf = None

for depth in range(1, 10):
    for quantity in range(1, 10):
        model_rf = RandomForestClassifier(random_state=12345,max_depth=depth,n_estimators=quantity) 
        model_rf.fit(features_train,target_train)
        predictions_valid = model_rf.predict(features_valid)
        result_rf = accuracy_score(predictions_valid, target_valid)
        print(f"Глубина: {depth}, Количество: {quantity}, Точность: {result_rf}")
        if result_rf > best_result_rf:
            best_model_rf = model_rf
            best_result_rf = result_rf
            best_depth_rf = depth
            best_quantity_rf = quantity

print(f"Лучшая модель: {best_model_rf}")
print(f"Лучшая точность: {best_result_rf}")
print(f"Лучшая глубина: {best_depth_rf}")
print(f"Лучшее количество: {best_quantity_rf}")

Глубина: 1, Количество: 1, Точность: 0.7607003891050583
Глубина: 1, Количество: 2, Точность: 0.7645914396887159
Глубина: 1, Количество: 3, Точность: 0.7607003891050583
Глубина: 1, Количество: 4, Точность: 0.7607003891050583
Глубина: 1, Количество: 5, Точность: 0.7607003891050583
Глубина: 1, Количество: 6, Точность: 0.7607003891050583
Глубина: 1, Количество: 7, Точность: 0.7587548638132295
Глубина: 1, Количество: 8, Точность: 0.7587548638132295
Глубина: 1, Количество: 9, Точность: 0.7587548638132295
Глубина: 2, Количество: 1, Точность: 0.7898832684824902
Глубина: 2, Количество: 2, Точность: 0.7879377431906615
Глубина: 2, Количество: 3, Точность: 0.7898832684824902
Глубина: 2, Количество: 4, Точность: 0.7937743190661478
Глубина: 2, Количество: 5, Точность: 0.7645914396887159
Глубина: 2, Количество: 6, Точность: 0.791828793774319
Глубина: 2, Количество: 7, Точность: 0.7782101167315175
Глубина: 2, Количество: 8, Точность: 0.7898832684824902
Глубина: 2, Количество: 9, Точность: 0.7782101167

### Обучим модель с использованием Дерева решений

In [14]:
best_result_dt = 0
best_depth_dt = 0
best_model_dt = None

for depth in range(1, 10):
    model_dt = DecisionTreeClassifier(random_state=12345,max_depth=depth)
    model_dt.fit(features_train,target_train)
    predictions_valid = model_dt.predict(features_valid)
    result_dt = accuracy_score(predictions_valid, target_valid)
    print(f"Глубина: {depth}, Точность: {result_dt}")
    
    if result_dt > best_result_dt:
        best_model_dt = model_dt
        best_result_dt = result_dt
        best_depth_dt = depth

print(f"Лучшая модель: {best_model_dt}")
print(f"Лучшая точность: {best_result_dt}")
print(f"Лучшая глубина: {best_depth_dt}")

Глубина: 1, Точность: 0.7431906614785992
Глубина: 2, Точность: 0.7840466926070039
Глубина: 3, Точность: 0.7879377431906615
Глубина: 4, Точность: 0.7801556420233463
Глубина: 5, Точность: 0.7879377431906615
Глубина: 6, Точность: 0.7898832684824902
Глубина: 7, Точность: 0.7859922178988327
Глубина: 8, Точность: 0.7821011673151751
Глубина: 9, Точность: 0.7821011673151751
Лучшая модель: DecisionTreeClassifier(max_depth=6, random_state=12345)
Лучшая точность: 0.7898832684824902
Лучшая глубина: 6


### Вывод
Лучшая точность у модели Случайного леса, больше 80% с глубиной 7 и количеством 6

## Проверьте модель на тестовой выборке

Используем данные лучшей модели

In [15]:
model = RandomForestClassifier(random_state=12345, n_estimators=best_quantity_rf,max_depth=best_depth_rf)
model.fit(features_train,target_train)
model.score(features_test, target_test)

0.813953488372093

## (бонус) Проверьте модели на адекватность

In [16]:
dummy = DummyClassifier(strategy='most_frequent', random_state=12345)
dummy.fit(features_train, target_train)
dummy_score = dummy.score(features_test, target_test)

print(f"Оценка точности случайной модели: {dummy_score}")

Оценка точности случайной модели: 0.7286821705426356


Точность у проверочной модели ниже чем у нашей

## Вывод
Таким образом компании Мегалайн для анализа и предложения клиентам нового тарифа следует использовать модель случайного леса. Эта модель имеет точность выше 80% и проходит проверку на адекватность 